# Resolving

Resolvers are helpers to find commonly used resources that one may want to link resources to. Resolvers are configured in the `Resolvers` section of the configuration file.

In [1]:
from kgforge.core import KnowledgeGraphForge

In [2]:
import getpass

In [3]:
token = getpass.getpass()

In [4]:
forge = KnowledgeGraphForge("../../configurations/demo-forge-nexus.yml", token=token)

# Imports

In [5]:
from kgforge.core.commons.strategies import ResolvingStrategy

## Terms

### scope
Resolve a resource for `female` in the terms.

In [6]:
female = forge.resolve("female", scope="terms")

In [7]:
type(female)

kgforge.core.resource.Resource

In [8]:
print(female)

{
    id: http://purl.obolibrary.org/obo/PATO_0000383
    label: female
}


### target

In [9]:
print(forge.resolve("female", scope="terms", target="sex"))

{
    id: http://purl.obolibrary.org/obo/PATO_0000383
    label: female
}


### type

In [10]:
print(forge.resolve("female", scope="terms", type="Class"))

{
    id: http://purl.obolibrary.org/obo/PATO_0000383
    label: female
}


## Entity

### scope
Resolving a resource for `EPFL` in the entities.

In [11]:
epfl = forge.resolve("EPFL", scope="entities")

In [12]:
type(epfl)

kgforge.core.resource.Resource

In [13]:
print(epfl)

{
    id: https://www.grid.ac/institutes/grid.5333.6
    label: École Polytechnique Fédérale de Lausanne
    acronym: EPFL
}


### target

In [14]:
print(forge.resolve("EPFL", scope="entities", target="agents"))

{
    id: https://www.grid.ac/institutes/grid.5333.6
    label: École Polytechnique Fédérale de Lausanne
    acronym: EPFL
}


### type

In [15]:
print(forge.resolve("EPFL", scope="entities", type="Organization"))

{
    id: https://www.grid.ac/institutes/grid.5333.6
    label: École Polytechnique Fédérale de Lausanne
    acronym: EPFL
}


## Strategies

Different strategies can be used to resolve terms. 

In the following example, the missing 'e' at the end is intended for the demonstration.

In [16]:
text = "mal"

### best match

The default applied strategy is `strategy=ResolvingStrategy.BEST_MATCH`.

In [17]:
print(forge.resolve(text, scope="terms"))

{
    id: http://purl.obolibrary.org/obo/PATO_0000384
    label: male
}


### exact match

In [18]:
print(forge.resolve(text, scope="terms", strategy=ResolvingStrategy.EXACT_MATCH))

None


### fuzzy match (all matches)

The result list is ordered by matching relevance.

In [19]:
results = forge.resolve(text, scope="terms", strategy=ResolvingStrategy.ALL_MATCHES)

In [20]:
type(results)

list

In [21]:
len(results)

2

In [22]:
type(results[0])

kgforge.core.resource.Resource

In [23]:
print(*results, sep="\n")

{
    id: http://purl.obolibrary.org/obo/PATO_0000384
    label: male
}
{
    id: http://purl.obolibrary.org/obo/PATO_0000383
    label: female
}
